In [8]:
%matplotlib inline

In [9]:
import sys
import os

# Add the project root directory to the Python path
# Assumes the notebook is in a subdirectory one level down from the root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
# Import the function
from TradeSim.testing_random_forest import initialize_test_account

# You can now use the function, for example:
# account = initialize_test_account()

AttributeError: 'ZMQInteractiveShell' object has no attribute 'magic'

In [ ]:
# logger = setup_logging("logs", "testing.log", level=logging.INFO)
# Initialize testing variables
account = initialize_test_account()

start_date = datetime.strptime(test_period_start, "%Y-%m-%d")
# test_period_end = "2024-12-04"
end_date = datetime.strptime(test_period_end, "%Y-%m-%d")
accuracy_threshold = 0.5
# account_values = pd.Series(index=pd.date_range(start=start_date, end=end_date))

# Create a US business day calendar
us_business_day = CustomBusinessDay(calendar=USFederalHolidayCalendar())
# Generate the date range using the custom business day calendar
test_date_range = pd.bdate_range(
    start=start_date, end=end_date, freq=us_business_day
)

logger.info(f"Testing period: {start_date} to {end_date}")

trades_list_db_name = os.path.join("PriceData", "trades_list_vectorised.db")
con_tl = sqlite3.connect(trades_list_db_name)
strategy_decisions_final_db_name = os.path.join(
    "PriceData", "strategy_decisions_final.db"
)
con_sd_final = sqlite3.connect(strategy_decisions_final_db_name)
predictions_db_name = os.path.join("PriceData", "predictions.db")
con_predictions = sqlite3.connect(predictions_db_name)
con_pd = sqlite3.connect(PRICE_DB_PATH)

strategies_list = get_tables_list(con_tl, logger)
# removes 'summary' table from list if it exists
if "summary" in strategies_list:
    strategies_list.remove("summary")
ticker_price_history = {}

logger.info(f"{test_period_start = } {test_period_end = }")
for ticker in train_tickers + regime_tickers:
    # query = f"SELECT * FROM '{ticker}' WHERE Date >= {test_period_start} AND Date <= {test_period_end}"
    # ticker_price_history[ticker] = pd.read_sql(query, con_pd, index_col=["Date"])

    query = f"SELECT * FROM '{ticker}' WHERE Date >= ? AND Date <= ?"
    ticker_price_history[ticker] = pd.read_sql(
        # query, con_pd, params=(start_date, end_date), index_col=["Date"]
        query,
        con_pd,
        params=(start_date, end_date),
        index_col=["Date"],
    )

# logger.info(f"{ticker_price_history = }")
# logger.info(f"{test_date_range = }")

strategies = [strategies[1]]
for date in test_date_range:
    prediction_results_list = []
    date_missing = False
    for idx, strategy in enumerate(strategies):
        strategy_name = strategy.__name__
        logger.info(f"{strategy_name} {idx + 1}/{len(strategies)}")

        # get strategy decisions from strategy_decisions_final.db
        query = f"SELECT * FROM {strategy_name}"
        precomputed_decisions = pd.read_sql(query, con_sd_final, index_col=["Date"])
        logger.info(f"{precomputed_decisions = }")

        # check if current date is in precomputed_decisions
        if date.strftime("%Y-%m-%d") not in precomputed_decisions.index:
            logger.warning(
                f"Date {date.strftime('%Y-%m-%d')} not found in precomputed decisions for {strategy_name}."
            )
            date_missing = True
            continue

        # load rf model
        if not check_model_exists(strategy_name):
            logger.error(f"Model for {strategy_name} does not exist, skipping...")
            continue
        rf_dict = load_rf_model(strategy_name, logger)
        if rf_dict is None:
            logger.error(
                f"Model for {strategy_name} could not be loaded, skipping..."
            )
            continue
        assert isinstance(
            rf_dict, dict
        ), "loaded_model is not a dictionary, model loading failed."
        # logger.info(f"{rf_dict}")

        for ticker in train_tickers:
            try:
                # Attempt to get the shifted decision
                action = precomputed_decisions.shift(1).loc[
                    date.strftime("%Y-%m-%d"), ticker
                ]
            except KeyError:
                # Log a warning and default action if the key is not found
                logger.warning(
                    f"Precomputed decision not found for {ticker} on {date.strftime('%Y-%m-%d')} after shift. Defaulting to 'hold'."
                )
                action = "hold"

            try:
                current_price = ticker_price_history[ticker].loc[
                    date.strftime("%Y-%m-%d")
                ]["Close"]
                current_price = round(current_price, 2)
            except KeyError:
                logger.warning(
                    f"Current price not found for {ticker} on {date.strftime('%Y-%m-%d')}."
                )
                continue

            logger.info(
                f"{ticker} - {date.strftime("%Y-%m-%d")} - {action} - current_price: {current_price}"
            )
            account = check_stop_loss_take_profit(account, ticker, current_price)
            # logger.info(f"{account = }")

            if action == "Buy":
                # daily_vix_df = ticker_price_history["^VIX"].loc[date_str]["Close"]
                daily_vix_df = ticker_price_history["^VIX"].loc[
                    date.strftime("%Y-%m-%d")
                ]["Close"]
                assert daily_vix_df is not None, "daily_vix_df is None"
                One_day_spy_return = ticker_price_history["^GSPC"].loc[
                    date.strftime("%Y-%m-%d")
                ]["One_day_spy_return"]
                assert One_day_spy_return is not None, "One_day_spy_return is None"

                data = {
                    "^VIX": [daily_vix_df],
                    "One_day_spy_return": [One_day_spy_return],
                }
                sample_df = pd.DataFrame(data, index=[0])

                """
                Load rf model. Pre-compute from strategy_decisions_intermediate.db. create new db with test dates?
                """

                prediction = predict_random_forest_classifier(
                    rf_dict["rf_classifier"],
                    sample_df,
                )

                if prediction != 1:
                    action = "hold"
                accuracy = rf_dict["accuracy"]
                logger.info(
                    f"Prediction {date.strftime("%Y-%m-%d")} {strategy_name} {ticker}: {prediction}, {accuracy = } vix: {daily_vix_df:.2f} spy:{One_day_spy_return:.2f} {action = }"
                )

                # weight = prediction * accuracy  # not needed
                prediction_results_list.append(
                    {
                        "strategy_name": strategy_name,
                        "ticker": ticker,
                        "action": action,
                        "prediction": prediction,
                        "accuracy": accuracy,
                        "current_price": current_price,
                    }
                )

                # execute sell orders

            if action == "sell":
                account = execute_sell_orders(
                    action,
                    ticker,
                    strategy_name,
                    account,
                    current_price,
                    date,
                    logger,
                )

    if not date_missing:

        # # Calculate holdings value by strategy. needed to ensure cash allocation constraints are met.
        # update in ticker loop?
        holdings_value_by_strategy = get_holdings_value_by_strategy(
            account, ticker_price_history, date
        )
        logger.info(f"{holdings_value_by_strategy = }")

        prediction_results_df = pd.DataFrame(prediction_results_list)
        logger.info(f"{prediction_results_df = }")

        buy_df = strategy_and_ticker_cash_allocation(
            prediction_results_df,
            account,
            holdings_value_by_strategy,
            prediction_threshold,
            train_trade_asset_limit,
            train_trade_strategy_limit,
        )

        buy_heap = create_buy_heap(buy_df)
        logger.info(f"{buy_heap = }")

        suggestion_heap = []
        # Execute buy orders, create sl and tp prices
        account = execute_buy_orders(
            buy_heap,
            suggestion_heap,
            account,
            date,
            train_trade_liquidity_limit,
            train_stop_loss,
            train_take_profit,
        )

        # Calculate and update account total portfolio value
        account = update_account_portfolio_values(
            account, ticker_price_history, date
        )
        logger.info(f"{account = }")

logger.info(f"{account['total_portfolio_value'] = }")

# Log daily results
logger.info("-------------------------------------------------")
logger.info(f"Account Cash: ${account['cash']: ,.2f}")
logger.info(f"Trades: {account['trades']}")
logger.info(f"Trades: {len(account['trades'])}")
logger.info(f"Holdings: {account['holdings']}")
logger.info(f"Total Portfolio Value: ${account['total_portfolio_value']: ,.2f}")
# logger.info(f"Active Count: {active_count}")
logger.info("-------------------------------------------------")
